# Potassium Calcium Channels

## 电生理特性

钙依赖性钾通道是一类在神经元中广泛存在的钾通道，其激活不仅依赖于膜电位，还依赖于胞内钙离子浓度的变化。

这类通道在神经元的兴奋-抑制调控、动作电位后超极化、放电频率调节等方面起着至关重要的作用。它们通常在钙离子内流后被激活，促进钾离子外流，从而使膜电位恢复或进入超极化状态。

## 建模实现

钙依赖性钾通道，在电生理学中经常简称为 `KCa` 通道，因此在建模时我们也沿用了这个写法，便于代码保持简洁性。

在 `braincell` 中，我们通过直接继承 `KCaChannel` 类来实现钠离子通道的建模。该类也是 `Channel` 的子类。

与前面介绍的通道一致，我们只需定义相应的接口就可以完成建模。

同样，我们还是给出一个例子。

`IAHP_De1994` 模型由 Destexhe 等人在 1994 年提出，其继承自 `KCaChannel` ，用于模拟慢性钙依赖钾通道的动力学行为，表达式如下：

$$
    \begin{aligned}
    (\text{closed}) + n \mathrm{Ca}_{i}^{2+} \underset{\beta}{\stackrel{\alpha}{\rightleftharpoons}} (\text{open})
    \end{aligned}
    $$

$$
\begin{aligned}
        I_{AHP} &= g_{\mathrm{max}} p^2 (V - E_K) \\
        {dp \over dt} &= \phi {p_{\infty}(V, [Ca^{2+}]_i) - p \over \tau_p(V, [Ca^{2+}]_i)} \\
        p_{\infty} &=\frac{\alpha[Ca^{2+}]_i^n}{\left(\alpha[Ca^{2+}]_i^n + \beta\right)} \\
        \tau_p &=\frac{1}{\left(\alpha[Ca^{2+}]_i +\beta\right)}
        \end{aligned}
$$

其中：
- $p$ 是通道状态变量
- $\alpha, \beta$ 是通道的开启/关闭速率常数
- $n$ 是钙离子结合数
- $\phi$ 是温度因子
- $g_{\text{max}}$ 是最大电导
- $E_K$ 是钾离子反转电位

代码实现如下：

```python
class IAHP_De1994(KCaChannel):
    def __init__(
        self,
        size: brainstate.typing.Size,
        n: Union[brainstate.typing.ArrayLike, Callable] = 2,
        g_max: Union[brainstate.typing.ArrayLike, Callable] = 10. * (u.mS / u.cm ** 2),
        alpha: Union[brainstate.typing.ArrayLike, Callable] = 48.,
        beta: Union[brainstate.typing.ArrayLike, Callable] = 0.09,
        phi: Union[brainstate.typing.ArrayLike, Callable] = 1.,
        name: Optional[str] = None,
    ):
        super().__init__(size=size, name=name)
        self.g_max = brainstate.init.param(g_max, self.varshape, allow_none=False)
        self.n = brainstate.init.param(n, self.varshape, allow_none=False)
        self.alpha = brainstate.init.param(alpha, self.varshape, allow_none=False)
        self.beta = brainstate.init.param(beta, self.varshape, allow_none=False)
        self.phi = brainstate.init.param(phi, self.varshape, allow_none=False)

    def init_state(self, V, K: IonInfo, Ca: IonInfo, batch_size=None):
        self.p = DiffEqState(brainstate.init.param(u.math.zeros, self.varshape, batch_size))

    def reset_state(self, V, K: IonInfo, Ca: IonInfo, batch_size=None):
        C2 = self.alpha * u.math.power(Ca.C / u.mM, self.n)
        C3 = C2 + self.beta
        if batch_size is None:
            self.p.value = u.math.broadcast_to(C2 / C3, self.varshape)
        else:
            self.p.value = u.math.broadcast_to(C2 / C3, (batch_size,) + self.varshape)
            assert self.p.value.shape[0] == batch_size

    def compute_derivative(self, V, K: IonInfo, Ca: IonInfo):
        C2 = self.alpha * u.math.power(Ca.C / u.mM, self.n)
        C3 = C2 + self.beta
        self.p.derivative = self.phi * (C2 / C3 - self.p.value) * C3 / u.ms

    def current(self, V, K: IonInfo, Ca: IonInfo):
        return self.g_max * self.p.value * self.p.value * (K.E - V)
```

其实这几个例子的建模方式高度统一，没有什么太大的区别。